In [1]:
import pandas as pd

# Define la ruta al nuevo dataset de La Liga
# Reemplaza 'datos_laliga/laliga_matches.csv' con la ruta y el nombre correctos
ruta_laliga = 'la_liga/matches_laliga.csv'

# Carga el archivo en un DataFrame de Pandas
try:
    df_laliga = pd.read_csv(ruta_laliga)
    print("Dataset de La Liga cargado exitosamente.")
    
    # Muestra las primeras 5 filas para un primer vistazo
    print("\nPrimeras 5 filas del dataset:")
    display(df_laliga.head())
    
    # Muestra información general sobre las columnas y los tipos de datos
    print("\nInformación general del dataset:")
    df_laliga.info()

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{ruta_laliga}'. Asegúrate de que el nombre de la carpeta y del archivo sean correctos.")

Dataset de La Liga cargado exitosamente.

Primeras 5 filas del dataset:


,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2025-08-16,19:30,La Liga,Matchweek 1,Sat,Away,W,3.0,0.0,...,Match Report,NaN,24.0,8.0,18.9,1.0,0,0,2025,Barcelona
1,1,2025-08-23,21:30,La Liga,Matchweek 2,Sat,Away,W,3.0,2.0,...,Match Report,NaN,26.0,10.0,17.0,1.0,0,0,2025,Barcelona
2,2,2025-08-31,21:30,La Liga,Matchweek 3,Sun,Away,D,1.0,1.0,...,Match Report,NaN,11.0,2.0,20.3,0.0,1,1,2025,Barcelona
3,3,2025-09-14,21:00,La Liga,Matchweek 4,Sun,Home,W,6.0,0.0,...,Match Report,NaN,24.0,10.0,18.4,0.0,0,0,2025,Barcelona
4,5,2025-09-21,21:00,La Liga,Matchweek 5,Sun,Home,W,3.0,0.0,...,Match Report,NaN,16.0,7.0,18.2,2.0,0,0,2025,Barcelona



Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 29 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     4700 non-null   int64  
 1   date           4700 non-null   object 
 2   time           4700 non-null   object 
 3   comp           4700 non-null   object 
 4   round          4700 non-null   object 
 5   day            4700 non-null   object 
 6   venue          4700 non-null   object 
 7   result         4700 non-null   object 
 8   gf             4700 non-null   float64
 9   ga             4700 non-null   float64
 10  opponent       4700 non-null   object 
 11  xg             4700 non-null   float64
 12  xga            4700 non-null   float64
 13  poss           4700 non-null   float64
 14  attendance     3724 non-null   float64
 15  captain        4700 non-null   object 
 16  formation      4700 non-null   object 
 17  opp formation  470

In [5]:
# Hacemos una copia para trabajar de forma segura
df_laliga_copy = df_laliga.copy()

# 1. Limpieza inicial
df_laliga_copy = df_laliga_copy.drop(columns=['Unnamed: 0', 'notes', 'match report', 'comp', 'round', 'day'])

# 2. Separamos el dataset
df_home = df_laliga_copy[df_laliga_copy['venue'] == 'Home'].copy()
df_away = df_laliga_copy[df_laliga_copy['venue'] == 'Away'].copy()

# 3. Renombramos las columnas (VERSIÓN CORREGIDA)
df_home = df_home.rename(columns={
    'result': 'result_home', 'poss': 'poss_home', # <-- 'poss' AÑADIDO AQUÍ
    'gf': 'gf_home', 'ga': 'ga_home', 'xg': 'xg_home', 'xga': 'xga_home', 
    'sh': 'sh_home', 'sot': 'sot_home', 'team': 'team_home', 'opponent': 'team_away'
})

df_away = df_away.rename(columns={
    'result': 'result_away', 'poss': 'poss_away', # <-- 'poss' AÑADIDO AQUÍ
    'gf': 'gf_away', 'ga': 'ga_away', 'xg': 'xg_away', 'xga': 'xga_away',
    'sh': 'sh_away', 'sot': 'sot_away', 'team': 'team_away', 'opponent': 'team_home'
})

# 4. LA GRAN FUSIÓN (MERGE)
df_laliga_final = pd.merge(
    df_home,
    df_away,
    on=['date', 'time', 'team_home', 'team_away', 'season', 'referee']
)

# 5. Creamos la variable objetivo 'winner'
df_laliga_final['winner'] = df_laliga_final.apply(
    lambda row: 2 if row['result_home'] == 'W' else (1 if row['result_home'] == 'D' else 0),
    axis=1
)

print("¡Dataset de La Liga transformado exitosamente!")
print(f"Número de partidos completos: {len(df_laliga_final)}")

¡Dataset de La Liga transformado exitosamente!
Número de partidos completos: 1461


In [6]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import joblib

# --- 1. DEFINICIÓN DE FEATURES CON EL NUEVO DATASET ---

# Seleccionamos las pistas más potentes del nuevo dataset de La Liga
# Incluimos goles esperados (xg), tiros (sh), tiros a puerta (sot) y posesión (poss)
features_laliga = [
    'xg_home', 'xga_home', 'poss_home', 'sh_home', 'sot_home',
    'xg_away', 'xga_away', 'poss_away', 'sh_away', 'sot_away'
]

# Preparamos los datos para el entrenamiento
# 'X' son las pistas, 'y' es el resultado
X = df_laliga_final[features_laliga]
y = df_laliga_final['winner']

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 2. ENTRENAMIENTO CON EL MEJOR MODELO (XGBOOST OPTIMIZADO) ---

print("Entrenando con XGBoost Optimizado y el nuevo dataset de La Liga...")

# Usamos la "mejor configuración" que GridSearchCV encontró para nosotros
best_params = {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}

xgb_model_final = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=3, 
    use_label_encoder=False, 
    eval_metric='mlogloss', 
    random_state=42,
    **best_params  # Aplicamos la configuración óptima
)

# Entrenamos el modelo con los nuevos y mejores datos
xgb_model_final.fit(X_train, y_train)
print("¡Modelo final entrenado!")


# --- 3. EVALUACIÓN FINAL ---
predictions = xgb_model_final.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"\n¡PRECISIÓN DEFINITIVA con Dataset Enriquecido: {accuracy * 100:.2f}%!")
print("\nReporte de Clasificación Final:")
print(classification_report(y_test, predictions, target_names=['Gana Visitante (0)', 'Empate (1)', 'Gana Local (2)']))


# --- 4. GUARDAR EL MODELO DEFINITIVO ---
# Guardamos este nuevo modelo como la versión final
joblib.dump(xgb_model_final, 'modelo_futbol_definitivo_v2.pkl')
joblib.dump(X_train, 'X_train_data_definitivo_v2.pkl')

print("\n¡Modelo definitivo (v2) y datos de entrenamiento guardados!")

Entrenando con XGBoost Optimizado y el nuevo dataset de La Liga...
¡Modelo final entrenado!

¡PRECISIÓN DEFINITIVA con Dataset Enriquecido: 61.77%!

Reporte de Clasificación Final:
                    precision    recall  f1-score   support

Gana Visitante (0)       0.66      0.64      0.65        98
        Empate (1)       0.44      0.30      0.36        77
    Gana Local (2)       0.65      0.81      0.72       118

          accuracy                           0.62       293
         macro avg       0.59      0.58      0.58       293
      weighted avg       0.60      0.62      0.60       293


¡Modelo definitivo (v2) y datos de entrenamiento guardados!


C:\Users\danie\ProyectoAndroid\ProyectoTitulo\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:06:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [7]:
## Exploración: ESPN Soccer Data

In [9]:
import os
# Reemplaza 'ruta/a/espn_data/base_data' con la ruta real
ruta_base_espn = 'ESPN/base_data' 
print(os.listdir(ruta_base_espn))

['dB_diagram.png', 'fixtures.csv', 'keyEventDescription.csv', 'leagues.csv', 'players.csv', 'standings.csv', 'status.csv', 'teamRoster.csv', 'teams.csv', 'teamStats.csv', 'venues.csv']


In [10]:
import pandas as pd

# Define la ruta a tu nuevo dataset de ESPN
# Reemplaza 'datos_espn' con el nombre de la carpeta que creaste
ruta_base_espn = 'ESPN/base_data'

# Carga el archivo de partidos (fixtures)
ruta_fixtures_espn = f"{ruta_base_espn}/fixtures.csv"

try:
    df_espn_fixtures = pd.read_csv(ruta_fixtures_espn)
    
    print("Dataset de partidos de ESPN cargado exitosamente.")
    print("\nPrimeras 5 filas:")
    display(df_espn_fixtures.head())

    print("\nInformación general del dataset:")
    df_espn_fixtures.info()

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{ruta_fixtures_espn}'.")

Dataset de partidos de ESPN cargado exitosamente.

Primeras 5 filas:


,Rn,seasonType,leagueId,eventId,date,venueId,attendance,homeTeamId,awayTeamId,homeTeamWinner,awayTeamWinner,homeTeamScore,awayTeamScore,homeTeamShootoutScore,awayTeamShootoutScore,statusId,updateTime
0,1,12136,3922,689519,2024-01-01 05:00:00,8680,61916,627,4396,True,False,5,0,0,0,28,2024-01-07 03:20:23
1,2,12136,3922,694555,2024-01-01 13:30:00,4775,0,658,1928,False,True,1,2,0,0,28,2024-01-07 03:20:24
2,3,12136,3922,693431,2024-01-02 13:00:00,7614,0,4895,2621,False,True,0,4,0,0,28,2024-01-07 03:20:59
3,4,12136,3922,694586,2024-01-04 12:00:00,9877,0,1928,6723,False,True,1,2,0,0,28,2024-01-07 03:21:28
4,5,12136,3922,690701,2024-01-04 13:30:00,9876,0,655,4388,True,False,1,0,0,0,28,2024-01-07 03:21:29



Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65300 entries, 0 to 65299
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rn                     65300 non-null  int64 
 1   seasonType             65300 non-null  int64 
 2   leagueId               65300 non-null  int64 
 3   eventId                65300 non-null  int64 
 4   date                   65300 non-null  object
 5   venueId                65300 non-null  int64 
 6   attendance             65300 non-null  int64 
 7   homeTeamId             65300 non-null  int64 
 8   awayTeamId             65300 non-null  int64 
 9   homeTeamWinner         65300 non-null  bool  
 10  awayTeamWinner         65300 non-null  bool  
 11  homeTeamScore          65300 non-null  int64 
 12  awayTeamScore          65300 non-null  int64 
 13  homeTeamShootoutScore  65300 non-null  int64 
 14  awayTeamShootoutScore  65300 non-nul

In [11]:
# La ruta a la carpeta base_data ya la tenemos
ruta_base_espn = 'ESPN/base_data'

# 1. Carga el archivo de estadísticas de equipo
ruta_stats_espn = f"{ruta_base_espn}/teamStats.csv"
try:
    df_espn_stats = pd.read_csv(ruta_stats_espn)
    print("Dataset de estadísticas de ESPN cargado exitosamente.")
    
    # 2. Une los dos DataFrames (partidos + estadísticas)
    # Usaremos la columna 'eventId' que es el identificador único de cada partido
    df_espn_completo = pd.merge(df_espn_fixtures, df_espn_stats, on='eventId')
    
    print("\n¡Fusión completada!")
    print(f"Número total de partidos con estadísticas: {len(df_espn_completo)}")
    
    # 3. Vistazo final
    print("\nPrimeras 5 filas del dataset fusionado de ESPN:")
    display(df_espn_completo.head())
    
    print("\nColumnas disponibles en el dataset fusionado:")
    print(df_espn_completo.columns.tolist())

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{ruta_stats_espn}'.")

Dataset de estadísticas de ESPN cargado exitosamente.

¡Fusión completada!
Número total de partidos con estadísticas: 90057

Primeras 5 filas del dataset fusionado de ESPN:


,Rn,seasonType_x,leagueId,eventId,date,venueId,attendance,homeTeamId,awayTeamId,homeTeamWinner,...,accurateLongBalls,longballPct,blockedShots,effectiveTackles,totalTackles,tacklePct,interceptions,effectiveClearance,totalClearance,updateTime_y
0,1,12136,3922,689519,2024-01-01 05:00:00,8680,61916,627,4396,True,...,27.0,0.6,10.0,9.0,16.0,0.6,22.0,10.0,10.0,2024-01-07 03:20:23
1,1,12136,3922,689519,2024-01-01 05:00:00,8680,61916,627,4396,True,...,19.0,0.3,3.0,12.0,16.0,0.8,12.0,37.0,37.0,2024-01-07 03:20:23
2,2,12136,3922,694555,2024-01-01 13:30:00,4775,0,658,1928,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-07 03:20:24
3,2,12136,3922,694555,2024-01-01 13:30:00,4775,0,658,1928,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-07 03:20:24
4,3,12136,3922,693431,2024-01-02 13:00:00,7614,0,4895,2621,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-07 03:20:59



Columnas disponibles en el dataset fusionado:
['Rn', 'seasonType_x', 'leagueId', 'eventId', 'date', 'venueId', 'attendance', 'homeTeamId', 'awayTeamId', 'homeTeamWinner', 'awayTeamWinner', 'homeTeamScore', 'awayTeamScore', 'homeTeamShootoutScore', 'awayTeamShootoutScore', 'statusId', 'updateTime_x', 'seasonType_y', 'teamId', 'teamOrder', 'possessionPct', 'foulsCommitted', 'yellowCards', 'redCards', 'offsides', 'wonCorners', 'saves', 'totalShots', 'shotsOnTarget', 'shotPct', 'penaltyKickGoals', 'penaltyKickShots', 'accuratePasses', 'totalPasses', 'passPct', 'accurateCrosses', 'totalCrosses', 'crossPct', 'totalLongBalls', 'accurateLongBalls', 'longballPct', 'blockedShots', 'effectiveTackles', 'totalTackles', 'tacklePct', 'interceptions', 'effectiveClearance', 'totalClearance', 'updateTime_y']


In [12]:
# Hacemos una copia para trabajar de forma segura
df_espn_copy = df_espn_completo.copy()

# 1. Separamos el dataset en dos: uno para los equipos locales y otro para los visitantes
# Usamos la columna 'teamId' y 'homeTeamId' para identificar las filas de los equipos locales
df_home_stats = df_espn_copy[df_espn_copy['teamId'] == df_espn_copy['homeTeamId']].copy()
df_away_stats = df_espn_copy[df_espn_copy['teamId'] == df_espn_copy['awayTeamId']].copy()

# 2. Renombramos las columnas de estadísticas en cada tabla
# Definimos las columnas que queremos renombrar
stats_cols = [
    'possessionPct', 'foulsCommitted', 'yellowCards', 'redCards', 'offsides', 
    'wonCorners', 'saves', 'totalShots', 'shotsOnTarget'
    # ... puedes añadir más estadísticas de la lista si las quieres usar
]

# Añadimos el sufijo '_home' a las estadísticas del equipo local
home_rename_dict = {col: f"{col}_home" for col in stats_cols}
df_home_stats.rename(columns=home_rename_dict, inplace=True)

# Añadimos el sufijo '_away' a las estadísticas del equipo visitante
away_rename_dict = {col: f"{col}_away" for col in stats_cols}
df_away_stats.rename(columns=away_rename_dict, inplace=True)

# 3. LA GRAN FUSIÓN (MERGE) de ESPN
# Seleccionamos las columnas que necesitamos de cada tabla
cols_base = ['eventId', 'date', 'homeTeamId', 'awayTeamId', 'homeTeamWinner', 'homeTeamScore', 'awayTeamScore']
cols_home = ['eventId'] + list(home_rename_dict.values())
cols_away = ['eventId'] + list(away_rename_dict.values())

# Unimos la tabla base con las stats del local
df_espn_final = pd.merge(df_espn_fixtures[cols_base], df_home_stats[cols_home], on='eventId')

# Unimos el resultado con las stats del visitante
df_espn_final = pd.merge(df_espn_final, df_away_stats[cols_away], on='eventId')


print("¡Dataset de ESPN transformado exitosamente!")
print(f"Número de partidos completos de ESPN: {len(df_espn_final)}")

print("\nPrimeras 5 filas del dataset final de ESPN:")
display(df_espn_final.head())

¡Dataset de ESPN transformado exitosamente!
Número de partidos completos de ESPN: 44468

Primeras 5 filas del dataset final de ESPN:


,eventId,date,homeTeamId,awayTeamId,homeTeamWinner,homeTeamScore,awayTeamScore,possessionPct_home,foulsCommitted_home,yellowCards_home,...,shotsOnTarget_home,possessionPct_away,foulsCommitted_away,yellowCards_away,redCards_away,offsides_away,wonCorners_away,saves_away,totalShots_away,shotsOnTarget_away
0,689519,2024-01-01 05:00:00,627,4396,True,5,0,58.9,7.0,0.0,...,10.0,41.1,6.0,2.0,0.0,1.0,3.0,6.0,8.0,1.0
1,694555,2024-01-01 13:30:00,658,1928,False,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,693431,2024-01-02 13:00:00,4895,2621,False,0,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,694586,2024-01-04 12:00:00,1928,6723,False,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,690701,2024-01-04 13:30:00,655,4388,True,1,0,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Hacemos una copia para no modificar el original
df_espn_limpio = df_espn_final.copy()

# Guardamos el número de filas antes de limpiar
filas_antes = len(df_espn_limpio)
print(f"Filas antes de la limpieza: {filas_antes}")

# Eliminamos cualquier fila que contenga al menos un valor nulo (NaN)
df_espn_limpio.dropna(inplace=True)

# Mostramos cuántas filas nos quedaron
filas_despues = len(df_espn_limpio)
print(f"Filas después de la limpieza: {filas_despues}")

# Verificación final: contamos los nulos de nuevo (deberían ser todos cero)
print("\nVerificación de nulos después de la limpieza:")
print(df_espn_limpio.isnull().sum())

Filas antes de la limpieza: 44468
Filas después de la limpieza: 24320

Verificación de nulos después de la limpieza:
eventId                0
date                   0
homeTeamId             0
awayTeamId             0
homeTeamWinner         0
homeTeamScore          0
awayTeamScore          0
possessionPct_home     0
foulsCommitted_home    0
yellowCards_home       0
redCards_home          0
offsides_home          0
wonCorners_home        0
saves_home             0
totalShots_home        0
shotsOnTarget_home     0
possessionPct_away     0
foulsCommitted_away    0
yellowCards_away       0
redCards_away          0
offsides_away          0
wonCorners_away        0
saves_away             0
totalShots_away        0
shotsOnTarget_away     0
dtype: int64


In [15]:
import numpy as np # <-- LÍNEA AÑADIDA
# Hacemos una copia para trabajar de forma segura
df_espn_estandarizado = df_espn_limpio.copy()

# 1. Crear la columna 'winner'
# El dataset de ESPN es más simple: homeTeamWinner es True o False (1 o 0)
# Como tu modelo de fútbol espera 3 resultados (2=Local, 1=Empate, 0=Visitante),
# crearemos la columna 'winner' basándonos en los goles.
df_espn_estandarizado['winner'] = np.where(
    df_espn_estandarizado['homeTeamScore'] > df_espn_estandarizado['awayTeamScore'], 2, 
    np.where(df_espn_estandarizado['homeTeamScore'] == df_espn_estandarizado['awayTeamScore'], 1, 0)
)


# 2. Renombrar las columnas de estadísticas para que coincidan
# Creamos un "diccionario de traducción"
column_mapper = {
    'shotsOnTarget_home': 'sot_home',
    'shotsOnTarget_away': 'sot_away',
    'totalShots_home': 'sh_home',
    'totalShots_away': 'sh_away',
    'wonCorners_home': 'corners_home',
    'wonCorners_away': 'corners_away'
    # NOTA: Este dataset de ESPN no tiene 'xg' (goles esperados) ni 'poss' (posesión).
    # Dejaremos esas columnas fuera por ahora para asegurar la compatibilidad.
}

df_espn_estandarizado.rename(columns=column_mapper, inplace=True)


print("¡Dataset de ESPN estandarizado exitosamente!")
print("\nNuevas columnas disponibles:")
print(df_espn_estandarizado.columns.tolist())

¡Dataset de ESPN estandarizado exitosamente!

Nuevas columnas disponibles:
['eventId', 'date', 'homeTeamId', 'awayTeamId', 'homeTeamWinner', 'homeTeamScore', 'awayTeamScore', 'possessionPct_home', 'foulsCommitted_home', 'yellowCards_home', 'redCards_home', 'offsides_home', 'corners_home', 'saves_home', 'sh_home', 'sot_home', 'possessionPct_away', 'foulsCommitted_away', 'yellowCards_away', 'redCards_away', 'offsides_away', 'corners_away', 'saves_away', 'sh_away', 'sot_away', 'winner']


In [16]:
# 1. Definimos las columnas comunes que queremos conservar.
#    Elegimos las estadísticas básicas que ambos datasets comparten.
columnas_comunes = [
    'winner', 
    'sot_home', 'sot_away', # Tiros a puerta
    'sh_home', 'sh_away'    # Tiros totales
]

# 2. Seleccionamos solo esas columnas en cada DataFrame
df_laliga_para_fusion = df_laliga_final[columnas_comunes]
df_espn_para_fusion = df_espn_estandarizado[columnas_comunes]

# 3. LA GRAN FUSIÓN (pd.concat)
#    Apilamos un dataset encima del otro para crear nuestro súper-dataset.
super_dataset = pd.concat([df_laliga_para_fusion, df_espn_para_fusion], ignore_index=True)

# 4. Verificación final
print("¡Fusión completada! Se ha creado el súper-dataset.")
print(f"Número total de partidos en el súper-dataset: {len(super_dataset)}")

print("\nInformación general del súper-dataset:")
super_dataset.info()

¡Fusión completada! Se ha creado el súper-dataset.
Número total de partidos en el súper-dataset: 25781

Información general del súper-dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25781 entries, 0 to 25780
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   winner    25781 non-null  int64  
 1   sot_home  25781 non-null  float64
 2   sot_away  25781 non-null  float64
 3   sh_home   25781 non-null  float64
 4   sh_away   25781 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1007.2 KB


In [17]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import joblib

# --- 1. PREPARACIÓN DE DATOS CON EL SÚPER-DATASET ---

# Definimos las features que tenemos en nuestro dataset fusionado
features = ['sot_home', 'sot_away', 'sh_home', 'sh_away']

# Preparamos los datos para el entrenamiento
X = super_dataset[features]
y = super_dataset['winner']

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 2. ENTRENAMIENTO CON EL MEJOR MODELO Y EL MEJOR DATASET ---

print("Entrenando con XGBoost Optimizado y el Súper-Dataset...")

# Usamos la "mejor configuración" que GridSearchCV encontró para nosotros
best_params = {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}

xgb_model_super = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=3, 
    use_label_encoder=False, 
    eval_metric='mlogloss', 
    random_state=42,
    **best_params  # Aplicamos la configuración óptima
)

# Entrenamos el modelo con el nuevo y masivo set de datos
xgb_model_super.fit(X_train, y_train)
print("¡Modelo final con Súper-Dataset entrenado!")


# --- 3. EVALUACIÓN FINAL ---
predictions = xgb_model_super.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"\n¡PRECISIÓN DEFINITIVA con Súper-Dataset: {accuracy * 100:.2f}%!")
print("\nReporte de Clasificación Final:")
print(classification_report(y_test, predictions, target_names=['Gana Visitante (0)', 'Empate (1)', 'Gana Local (2)']))


# --- 4. GUARDAR EL MODELO DEFINITIVO ---
# Si este modelo es el mejor, lo guardamos
joblib.dump(xgb_model_super, 'modelo_futbol_super.pkl')
joblib.dump(X_train, 'X_train_data_super.pkl')

print("\n¡Súper-modelo y datos de entrenamiento guardados!")

Entrenando con XGBoost Optimizado y el Súper-Dataset...
¡Modelo final con Súper-Dataset entrenado!

¡PRECISIÓN DEFINITIVA con Súper-Dataset: 59.39%!

Reporte de Clasificación Final:
                    precision    recall  f1-score   support

Gana Visitante (0)       0.59      0.63      0.61      1585
        Empate (1)       0.45      0.12      0.19      1297
    Gana Local (2)       0.61      0.84      0.71      2275

          accuracy                           0.59      5157
         macro avg       0.55      0.53      0.50      5157
      weighted avg       0.56      0.59      0.55      5157


¡Súper-modelo y datos de entrenamiento guardados!


C:\Users\danie\ProyectoAndroid\ProyectoTitulo\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:40:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [18]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import joblib
import pandas as pd
import numpy as np

# --- 1. INGENIERÍA DE CARACTERÍSTICAS SOBRE EL DATASET DE CALIDAD ---

# Usamos el df_laliga_final que ya transformamos
df_laliga_limpio = df_laliga_final.copy()
df_laliga_limpio['date'] = pd.to_datetime(df_laliga_limpio['date'])
df_laliga_limpio = df_laliga_limpio.sort_values(by='date')

# A. Creación de Features de "Forma" (Rolling Averages)
# (Usaremos las estadísticas de FBref que son más ricas)
rolling_features = ['xg_home', 'sot_home', 'poss_home']
rolling_features_away = ['xg_away', 'sot_away', 'poss_away']

for feature in rolling_features:
    new_feature_name = f"rolling_avg_{feature}"
    df_laliga_limpio[new_feature_name] = df_laliga_limpio.groupby('team_home')[feature].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=1).mean()
    )

for feature in rolling_features_away:
    new_feature_name = f"rolling_avg_{feature}"
    df_laliga_limpio[new_feature_name] = df_laliga_limpio.groupby('team_away')[feature].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=1).mean()
    )

# B. Creación de Features Diferenciales
df_laliga_limpio['xg_diff'] = df_laliga_limpio['rolling_avg_xg_home'] - df_laliga_limpio['rolling_avg_xg_away']
df_laliga_limpio['sot_diff'] = df_laliga_limpio['rolling_avg_sot_home'] - df_laliga_limpio['rolling_avg_sot_away']
df_laliga_limpio['poss_diff'] = df_laliga_limpio['rolling_avg_poss_home'] - df_laliga_limpio['rolling_avg_poss_away']

df_laliga_limpio.dropna(inplace=True)
print("Nuevas features avanzadas creadas sobre el dataset de La Liga.")

# --- 2. ENTRENAMIENTO CON TODAS LAS TÉCNICAS ---

features_finales = [
    'xg_home', 'xga_home', 'poss_home', 'sh_home', 'sot_home',
    'xg_away', 'xga_away', 'poss_away', 'sh_away', 'sot_away',
    'rolling_avg_xg_home', 'rolling_avg_sot_home', 'rolling_avg_poss_home',
    'rolling_avg_xg_away', 'rolling_avg_sot_away', 'rolling_avg_poss_away',
    'xg_diff', 'sot_diff', 'poss_diff'
]

X = df_laliga_limpio[features_finales]
y = df_laliga_limpio['winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nEntrenando el modelo definitivo (XGBoost Optimizado + Features Avanzadas)...")

best_params = {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}

# Técnica para balancear clases en XGBoost
# Contamos cuántas veces aparece cada resultado
counts = y_train.value_counts()
scale_pos_weight_empate = counts[0] / counts[1] # Ratio para la clase minoritaria (empate)

xgb_definitivo = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=3, 
    eval_metric='mlogloss', 
    random_state=42,
    # scale_pos_weight=scale_pos_weight_empate, # Descomentar si el de arriba no funciona bien para multiclase
    **best_params
)

xgb_definitivo.fit(X_train, y_train)
print("¡Modelo definitivo entrenado!")

# --- 3. EVALUACIÓN FINAL ---
predictions = xgb_definitivo.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"\n¡PRECISIÓN DEFINITIVA: {accuracy * 100:.2f}%!")
print("\nReporte de Clasificación Final:")
print(classification_report(y_test, predictions, target_names=['Gana Visitante (0)', 'Empate (1)', 'Gana Local (2)']))

Nuevas features avanzadas creadas sobre el dataset de La Liga.

Entrenando el modelo definitivo (XGBoost Optimizado + Features Avanzadas)...
¡Modelo definitivo entrenado!

¡PRECISIÓN DEFINITIVA: 58.67%!

Reporte de Clasificación Final:
                    precision    recall  f1-score   support

Gana Visitante (0)       0.61      0.61      0.61        64
        Empate (1)       0.33      0.25      0.29        55
    Gana Local (2)       0.67      0.75      0.71       106

          accuracy                           0.59       225
         macro avg       0.53      0.54      0.53       225
      weighted avg       0.57      0.59      0.58       225

